In [ ]:
import pathlib
import astropy.coordinates as coord
from astropy.stats import median_absolute_deviation as MAD
import astropy.table as at
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import h5py
from tqdm import tqdm
from sklearn.neighbors import KDTree
from pyia import GaiaData
from scipy.stats import binned_statistic

from helpers import load_data, Features

Load APOGEE x Gaia data — see `Assemble-data.ipynb` for more information.

In [ ]:
# Upper giant branch:
g = load_data(
    filters=dict(
        TEFF=(3000, 5300), 
        LOGG=(-0.5, 2.3),
        M_H=(-3, None),
        phot_bp_mean_mag=(None, 15*u.mag),
        TEFF_ERR=(None, 100)
    )
)
len(g)

In [ ]:
bprp = (g.phot_bp_mean_mag - g.phot_rp_mean_mag).value
mg = (g.phot_g_mean_mag - g.get_distance(allow_negative=True).distmod).value

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

ax = axes[0]
H, xb, yb, _ = ax.hist2d(
    g.TEFF,
    g.LOGG,
    bins=(
        np.linspace(3000, 8000, 128),
        np.linspace(-0.5, 5.5, 128)
    ),
    norm=mpl.colors.LogNorm()
)
ax.set_xlim(xb.max(), xb.min())
ax.set_ylim(yb.max(), yb.min())
ax.set_xlabel('TEFF')
ax.set_ylabel('LOGG')

ax = axes[1]
H, xb, yb, _ = ax.hist2d(
    bprp,
    mg,
    bins=(
        np.linspace(-0.5, 3, 128),
        np.linspace(-4, 10.5, 128)
    ),
    norm=mpl.colors.LogNorm()
)
ax.set_xlim(xb.min(), xb.max())
ax.set_ylim(yb.max(), yb.min())
ax.set_xlabel('BP-RP')
ax.set_ylabel('$M_G$')

fig.tight_layout()

In [ ]:
label_name = 'TEFF'
label_y = g[label_name]
label_yerr = g[label_name + '_ERR']

TODO: binned_statistic on errorbar distributions

In [ ]:
fig, axes = plt.subplots(
    5, 5, 
    figsize=(12, 12),
    sharex=True, 
    constrained_layout=True
)

for i, ax in enumerate(axes.flat, start=1):
    val = g.bp[:, i] / g.bp[:, 0]
    val_err = val * np.sqrt((g.bp_err[:, i] / g.bp[:, i])**2 + (g.bp_err[:, 0] / g.bp[:, 0])**2)
    
    ax.errorbar(
        label_y,
        val,
        xerr=label_yerr,
        yerr=val_err,
        ls='none', marker='o', mew=0, ms=1,
        elinewidth=0.75, ecolor='tab:blue',
        alpha=0.5
    )
    
    med = np.nanpercentile(val, 50)
    lim = np.nanpercentile(val, [16, 84])
    lim = 3 * (lim[1] - lim[0])/2
    ax.set_ylim(med - lim, med + lim)
    
    ax.yaxis.set_visible(False)
    
fig.suptitle('BP', fontsize=24)

In [ ]:
fig, axes = plt.subplots(
    5, 5, 
    figsize=(12, 12),
    sharex=True, 
    constrained_layout=True
)

for i, ax in enumerate(axes.flat, start=1):
    val = g.rp[:, i] / g.rp[:, 0]
    val_err = val * np.sqrt((g.rp_err[:, i] / g.rp[:, i])**2 + (g.rp_err[:, 0] / g.rp[:, 0])**2)
    
    ax.errorbar(
        label_y,
        val,
        xerr=label_yerr,
        yerr=val_err,
        ls='none', marker='o', mew=0, ms=1,
        elinewidth=0.75, ecolor='tab:blue',
        alpha=0.5
    )
    
    med = np.nanpercentile(val, 50)
    lim = np.nanpercentile(val, [16, 84])
    lim = 3 * (lim[1] - lim[0])/2
    ax.set_ylim(med - lim, med + lim)
    
    ax.yaxis.set_visible(False)
    
fig.suptitle('RP', fontsize=24)